In [232]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *

In [233]:
rules = pd.read_csv(config.files['service_rules'])

In [234]:
class GroupedDataRegex:
    
    def __init__(
        self, joiner = '|'
    ):
        self.concat_char = joiner
        
    def make(self, groupby):
        return groupby.apply(
            lambda t: self.concat_char.join([str(i) for i in t])
        ).astype(str)

class GroupedDataRegexDecorator:
    
    def __init__(
        self, regexer, param, prefix, suffix
    ):
        self.regexer = regexer
        self.param = param
        self.prefix = prefix
        self.suffix = suffix
        
    def getType(self):
        return self.param
    
    def make(self, groupby):
        return (
            self.prefix
            + self.regexer.make(groupby)
            + self.suffix
        )



In [235]:
matchers = []
regexer = GroupedDataRegex('|')

rdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'M', '(', ')'))
sdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'S', '^(', ')$'))
adecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'A', '^.*(', ').*$'))

copydd = rules[(rules['text_match'].notna())]

regexes = {}
for matcher in matchers:
    type = copydd['matching']
    regex = copydd[copydd['matching'] == matcher.getType()][['service_name','text_match']]
    regex = matcher.make(regex.groupby('service_name')['text_match']).to_dict()
    inv_map = {v: k for k, v in regex.items()}
    regexes = {**regexes, **inv_map}
    

output(regexes)


{'^(CBS |CFS|Lim Commercials 3mo|CBS - Fake Cancel|CBS All Access|Commercial Free 1 Week|CBS - Fake New|CBS All Access|CBS - Fake Cancel|Cancellation Confirmation CBS|CBS - Fake Cancel Passive Churn)$': 'CBS '
                                                                                                                                                                                                           'All '
                                                                                                                                                                                                           'Access',
 '^(Netflix|Basic (Netflix)|Premium (Netflix)|Standard (Netflix)|Netflix - Fake Cancel Passive Churn|Netflix)$': 'Netflix',
 '^(SHOWTIME|SHOWTIME - Fake New|SHOWTIME|Showtime Subscription|Cancellation Confirmation SHOWTIME|SHOWTIME - Fake Cancel|SHOWTIME - Fake Cancel|CBS All Access and Showtime|Showtime 1 month free trial|Showtime|SHOWTIME - Fake New|SHOWTIME - Fa